In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install datasets
!pip uninstall torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install --upgrade transformers datasets
!pip cache purge

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer



# 1. Excel'den verileri yükle
file_path = r'/content/drive/My Drive/akış-diyagramı-örnekleri-verileri.xlsx'
veriler = pd.read_excel(file_path)

# Giriş ve çıkış verilerini ayıkla
veriler['input_text'] = veriler['Akış Cümlesi']  # Giriş verisi
veriler['output_text'] = veriler.apply(
    lambda row: f"{row['Adım No']} {row['Adım Açıklaması']} {row['Adım Türü']} {row['Koşul (Varsa)']} {row['Evet Durumu (Sonraki Adım)']} {row['Hayır Durumu (Sonraki Adım)']}", axis=1
)

#veriler = veriler.rename(columns={"Akış Cümlesi": "input_text", "Çıkış Verisi": "output_text"})
#veriler["output_text"] = veriler["output_text"].astype(str)  # Çıkışı metne dönüştür

veriler["Evet Durumu (Sonraki Adım)"] = veriler["Evet Durumu (Sonraki Adım)"].apply(lambda x: None if isinstance(x, str) and x.strip() == '-' else x)
veriler["Hayır Durumu (Sonraki Adım)"] = veriler["Evet Durumu (Sonraki Adım)"].apply(lambda x: None if isinstance(x, str) and x.strip() == '-' else x)

# Veri önizleme
print("Veri önizleme:", veriler.head())

# 2. Dataset formatına dönüştür
dataset = Dataset.from_pandas(veriler)

# 3. Model ve Tokenizer yükle
model_name = "t5-small"  # Küçük bir model kullanıyoruz
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


#tokenized_datasets = dataset.map(preprocess_function, batched=True)
# 4. Tokenizasyon işlemi
def preprocess_function(examples):
    inputs = [str(x) for x in examples["input_text"]]  # Giriş cümleleri
    targets = [str(x) for x in examples["output_text"]]  # Çıkış detayları
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=512, truncation=True, padding='max_length').input_ids   # Çıkışlar daha uzun olabilir
    model_inputs["labels"] = labels
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

# 5. Eğitim ayarları
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    logging_dir="./logs"
)

from sklearn.model_selection import train_test_split

# Eğitim ve doğrulama verilerini ayırma

train_dataset = tokenized_datasets.train_test_split(test_size=0.1, seed=42)["train"]
eval_dataset = tokenized_datasets.train_test_split(test_size=0.1, seed=42)["test"]

# 6. GPU kontrolü ve modele aktarım
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Modeli GPU'ya taşı

# 6. Eğitici (Trainer) tanımlama
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Doğrulama veri kümesini ekleyin
    tokenizer=tokenizer
)

# 6. Eğitici (Trainer) tanımlama
#trainer = Trainer(
 #   model=model,
 #   args=training_args,
 #   train_dataset=tokenized_datasets,
 #   tokenizer=tokenizer
#)

# 7. Modeli eğit
trainer.train()

# 8. Tahmin testi
def predict(cumle):
    inputs = tokenizer(cumle, return_tensors="pt", max_length=128, truncation=True)
    outputs = model.generate(**inputs, max_length=512)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test için örnek bir cümle
test_cumle = "Sipariş bilgilerini al, stokta ürün var mı kontrol et, varsa siparişi kaydet."
tahmin = predict(test_cumle)
print("Tahmin edilen adımlar ve türleri:", tahmin)


In [ ]:
#Kayıpları izlemek için
# Eğitim ve doğrulama kaybını kontrol et
logs = trainer.state.log_history  # Eğitim loglarını al
train_loss = [log['loss'] for log in logs if 'loss' in log]
eval_loss = [log['eval_loss'] for log in logs if 'eval_loss' in log]

print("Eğitim Kayıpları (Train Loss):", train_loss)
print("Doğrulama Kayıpları (Eval Loss):", eval_loss)


In [ ]:
#Görsel analiz için
import matplotlib.pyplot as plt

# Kayıpları çiz
plt.plot(train_loss, label="Eğitim Kaybı")
plt.plot(eval_loss, label="Doğrulama Kaybı")
plt.xlabel("Epoch")
plt.ylabel("Kayıp (Loss)")
plt.title("Eğitim ve Doğrulama Kayıpları")
plt.legend()
plt.show()
